In [ ]:
import whitebox
import os
import shutil
import fiona 
import geopandas as gpd
import matplotlib.pyplot as plt

In [ ]:
##### initialize whitebox

# import whitebox tools class as wbt object
wbt = whitebox.WhiteboxTools()

# toggle on/off geoprocessing tool outputs
# wbt.verbose = False

# set working directory for input/output files to current working directory
wbt.set_working_dir(os.getcwd())

# print whitebox version to verify correctly loading
wbt.version()

# NOTE: original terrain features calculated in ArcGIS Pro, but can be done using Whitebox

# DEM Pre-Processing

In [ ]:
###### copy original dem to current working directory - whitebox doesn't like different directories

# path to original dem
original_dem_path = r'../Data/dem_10m/dem_10m_clipped_26916.tif'

# path to copy original dem (current workind directory plus same basename)
copy_dem_path = os.path.join(r'terrain_features', os.path.basename(original_dem_path))

# check that original dem does exist
if os.path.exists(original_dem_path):

    # check that copy dem does NOT already exist
    if not os.path.exists(copy_dem_path):

        # copy original dem to new location
        shutil.copy(original_dem_path, copy_dem_path)

    else:
        print('Copy DEM already exists...')

else:
    print('Original DEM does not exist in specified path!')

## Feature Preserving Smoothing

In [ ]:
##### https://www.whiteboxgeo.com/manual/wbt_book/available_tools/geomorphometric_analysis.html#FeaturePreservingSmoothing

# path to copied mosaic and clipped dem in UTM projection - whitebox 2.3.0 does NOT like relative paths for input!!!
input_dem = os.path.abspath(r'terrain_features/dem_10m_clipped_26916.tif')

# path for smoothed output dem - whitebox 2.3.0 does NOT like relative paths for output!!!
output_dem = os.path.abspath(r'terrain_features/dem_fps.tif')


# check that smooth dem does not already exist
if not os.path.exists(output_dem):

    # smooth dem for further processing (default params: filter=11, norm_diff=15, num_iter=3, max_diff=0.5)
    wbt.feature_preserving_smoothing(dem=input_dem, output=output_dem, filter=5, norm_diff=5, num_iter=3)

else:
    print('DEM has already been smoothed and exists...')

## Hydrological Integrity

### Get Streams

In [ ]:
# path to geodatabase
gdb_path = r'../Data/nhdplus_hr/KY_NHDPlus_H_National_Release_1_26916.gdb'

# list all layers to find correct feature class name
layers = fiona.listlayers(gdb_path)
for layer in layers:
    print(layer)

In [ ]:
# read flowlines into geodataframe; specify feature class name from list above
flowline_features = gpd.read_file(gdb_path, layer='KY_NetworkNHDFlowline')

# display geodataframe head
flowline_features.head()

In [ ]:
flowline_features.info()

In [ ]:
flowline_features['streamorde'].value_counts()

In [ ]:
fig, ax = plt.subplots(figsize=(12,7))

flowline_features.plot(column='streamorde', 
                       categorical=True, 
                       linewidth=0.4, 
                       ax=ax, 
                       label='Stream Order', 
                       legend=True, 
                       legend_kwds={'title':'Stream Order', 'frameon':False,
                                    'bbox_to_anchor':(0,1), 'loc':'upper left'})

ax.set_axis_off()

plt.show()

In [ ]:
minimum_stream_order = 3

for minimum_stream_order in range(2,5):
    
    fig, ax = plt.subplots(figsize=(12,7))
    
    flowline_features[flowline_features['streamorde'] >= minimum_stream_order].plot(linewidth=0.5, ax=ax)
    
    ax.set_axis_off()
    ax.set_title(f"Streams \u2265 {minimum_stream_order} stream order", y=0.9, loc='left')
    
    plt.show()

In [ ]:
minimum_stream_order_threshold = 3

flowline_shapefile_output = f"terrain_features/nhdplus_hr_order{minimum_stream_order_threshold}_26916.shp"

if not os.path.exists(flowline_shapefile_output):
    mask = flowline_features['streamorde'] >= 3
    flowline_features.loc[mask, ['gnis_id', 'streamorde', 'geometry']].to_file(flowline_shapefile_output)

else:
    print('Shapefile already exists...')

### Burn Streams in DEM

In [ ]:
# burn streams in smoothed dem...

input_dem = os.path.abspath(r'terrain_features/dem_fps.tif')
input_flowlines = os.path.abspath(r'terrain_features/nhdplus_hr_order3_26916.shp')

output_dem = os.path.abspath(r'terrain_features/dem_fps_burned.tif')


# out_flowline = os.path.abspath(r'terrain_features/stream_prunednetwork.tif')
# out_flowdirection = os.path.abspath(r'terrain_features/stream_flowdirection.tif')
# out_flowaccumulation = os.path.abspath(r'terrain_features/stream_flowaccumulation.tif')

# wbt.topological_breach_burn(
#     streams=flowline_path, 
#     dem=dem_path, 
#     out_streams=out_flowline, 
#     out_dem=out_dem, 
#     out_dir=out_flowdirection, 
#     out_accum=out_flowaccumulation, 
#     snap=1.0, 
#     callback=default_callback
# )


# check that smooth dem does not already exist
if not os.path.exists(output_dem):

    # burn streams in smoothed DEM
    wbt.fill_burn(dem=input_dem, streams=input_flowlines, output=output_dem)

else:
    print('DEM has already been burned and exists...')

In [ ]:
# fill single cell depressions (final pre-processing step)

input_dem = os.path.abspath(r'terrain_features/dem_fps_burned.tif')
output_dem = os.path.abspath(r'terrain_features/dem_fps_burned_singlecellfill.tif')


# check that smooth dem has not already been processed
if not os.path.exists(output_dem):

    # fill single-cell pits/depressions in smoothed & burned dem
    wbt.fill_single_cell_pits(dem=input_dem, output=output_dem)


else:
    print('DEM has already been filled and exists...')

# Terrain Features

In [ ]:
# path to pre-processed DEM (smoothed, stream burn, single-cell fill)
dem_input_path = os.path.abspath(r'terrain_features/dem_fps_burned_singlecellfill.tif')

## Slope

In [ ]:
# calculate slope (degrees)

output_path = os.path.abspath(r'terrain_features/slope_deg.tif')

wbt.slope(
    dem=dem_input_path, 
    output=output_path, 
    zfactor=None, 
    units='degrees', 
)

In [ ]:
# add small slope so nothing is completely flat?

## Aspect

In [ ]:
# calculate aspect (0-360 degrees); shouldn't be any flat/undefined areas because added small slope

output_path = os.path.abspath(r'terrain_features/aspect.tif')

wbt.aspect(
    dem, 
    output, 
    zfactor=None, 
    callback=default_callback
)

## Profile Curvature

## Tangential Curvature

## Roughness Index

In [ ]:
# multiple windows

## Standard Devation of Elevation

In [ ]:
# multiple windows

## Topographic Postion Index

In [ ]:
# multiple windows

## Topographic Wetness Index

## Stream Power Index

## Drainage Density (Watershed measurement)